In [ ]:
                        ### Detección dePatrones de Velas Japonesas usando TALib ###

import pandas as pd
import numpy as np
import talib
import mplfinance as mpf

# Supongamos que tu DataFrame se llama 'data' y tiene índices de fecha
# y columnas 'Open', 'High', 'Low', 'Close'.

# Detectar varios patrones de velas japonesas
patrones = {
    'Doji': talib.CDLDOJI,
    'Martillo': talib.CDLHAMMER,
    'Estrella del Amanecer': talib.CDLMORNINGSTAR,
    'Hombre Colgado': talib.CDLHANGINGMAN,
    'Envolvente Alcista': talib.CDLENGULFING,
    'Tres Cuervos Negros': talib.CDLIDENTICAL3CROWS,
    'Estrella Fugaz': talib.CDLSHOOTINGSTAR,
    'Tres Soldados Blancos': talib.CDL3WHITESOLDIERS
}

# Colores para cada patrón
colores = {
    'Doji': 'blue',
    'Martillo': 'green',
    'Estrella del Amanecer': 'purple',
    'Hombre Colgado': 'red',
    'Envolvente Alcista': 'lime',
    'Tres Cuervos Negros': 'black',
    'Estrella Fugaz': 'orange',
    'Tres Soldados Blancos': 'cyan'
}

# Calcular cada patrón y almacenarlo en el DataFrame
for nombre, funcion in patrones.items():
    data[nombre] = funcion(data['Open'], data['High'], data['Low'], data['Close'])

# Preparar los addplots para cada patrón
addplots = []
for nombre, color in colores.items():
    # Para patrones alcistas, colocar marcador debajo de la vela
    if nombre in ['Martillo', 'Estrella del Amanecer', 'Envolvente Alcista', 'Tres Soldados Blancos']:
        # Colocar 3% por debajo del mínimo para patrones alcistas
        signals = data['Low'].where(data[nombre] > 0, np.nan) * 0.97
    else:
        # Colocar 3% por encima del máximo para patrones bajistas
        signals = data['High'].where(data[nombre] > 0, np.nan) * 1.03
    
    addplot = mpf.make_addplot(
        signals,
        type='scatter',
        marker='^' if nombre in ['Martillo', 'Estrella del Amanecer', 'Envolvente Alcista', 'Tres Soldados Blancos'] else 'v',
        markersize=100,
        color=color,
        marker_label=nombre
    )
    addplots.append(addplot)

# Configuración para estilo personalizado
mi_estilo = mpf.make_mpf_style(
    marketcolors=mpf.make_marketcolors(
        up='green', down='red',
        wick={'up':'green', 'down':'red'},
        volume='blue'
    ),
    gridstyle=':',
    y_on_right=False
)

# Graficar la vela japonesa con mplfinance
fig, axes = mpf.plot(
    data, 
    type='candle', 
    addplot=addplots,
    style=mi_estilo,
    title='Gráfico de Velas con Patrones Japoneses',
    volume=True,
    figsize=(15, 10),
    panel_ratios=(4, 1),  # Proporción entre el gráfico principal y el volumen
    returnfig=True        # Para poder añadir una leyenda posteriormente
)

# Añadir leyenda
ax = axes[0]
ax.legend(
    [ap.lines[0] for ap in addplots],
    [ap.kwargs.get('marker_label', '') for ap in addplots],
    loc='best',
    frameon=True,
    framealpha=0.9
)

fig.savefig('patrones_velas.png')  # Guardar la figura
plt.show()